In [1]:
import os
import pickle

In [2]:
!pip install -qq transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires nvidia-ml-py3, which is not installed.


In [9]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from bs4 import BeautifulSoup
import torch.nn.functional as F

In [15]:
from dataloader_classes_func import ReviewDataset, createDataLoader

In [5]:
with open("./data/train_reviews_v2.pkl",'rb') as fp:
    train_reviews= pickle.load(fp)

with open("./data/val_reviews_v2.pkl",'rb') as fp:
    val_reviews= pickle.load(fp)
    
with open("./data/train_labels_v2.pkl",'rb') as fp:
    train_labels= pickle.load(fp)

with open("./data/val_labels_v2.pkl",'rb') as fp:
    val_labels= pickle.load(fp)

In [6]:
MODEL_NAME = 'bert-base-uncased'
tokenizer= BertTokenizer.from_pretrained(MODEL_NAME)

In [7]:
train_dataset= {'data': train_reviews, 'labels': train_labels}
train_dataloader= createDataLoader(train_dataset, tokenizer)

In [10]:
class SentimentClassifier(torch.nn.Module):
    
    def __init__(self, n_class=2):
        super(SentimentClassifier, self).__init__()
        self.n_class= n_class
        self.bert_model= BertModel.from_pretrained('bert-base-cased')
        self.drop = torch.nn.Dropout(p=0.3)
        self.out= torch.nn.Linear(self.bert_model.config.hidden_size, self.n_class)
        
    def forward(self, input_ids, attention_mask):
        bert_output= self.bert_model(input_ids= input_ids, attention_mask= attention_mask)
        dropout_output= self.drop(bert_output.pooler_output)
        output= F.softmax(dropout_output, dim= 1)
        return output

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
model= SentimentClassifier()
model= model.to(device)

In [16]:
dl_item= iter(train_dataloader).next()
ip_id_tensor= dl_item['encoding']['input_ids'].to(device)
attention_mask_tensor= dl_item['encoding']['attention_mask'].to(device)

NameError: name 'tokenizer' is not defined

In [11]:
!nvidia-smi

Wed Feb  3 10:36:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   28C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------